## Test indexer

### Global

In [1]:
# from langchain_openai import ChatOpenAI
# from langchain_google_genai import ChatGoogleGenerativeAI
import uuid
from functools import partial
import datetime
import uuid
from big_seo.common.lang import DocumentLang

In [2]:
start = datetime.datetime.now()
def get_unique_doc(res):
    for doc in res:
        doc[1]['doc'].score = doc[0]
    return set([doc[1]['doc'] for doc in res])

In [3]:
from big_seo.crawler.api import Crawler, UpToDateCrawler
from big_seo.llm.implement.indexer import WebPageIndexer, InPageIndexer, UpToDateInPageIndexer
from big_seo.llm.implement.parser import WebPageParser, TranslationParser, UpToDateWebPageParser
from big_seo.llm.implement.data_model import WebPageDocument, OutlineDocumentDBType, Base, UpToDateCollection
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from big_seo.llm.implement.embedding_model import LLamaIndexEmbedding
from big_seo.llm.implement.parser import OutlineDocumentCreator


from big_seo.translator.core import GeminiHealthcareTranslator


from langchain_openai import ChatOpenAI
from langchain_google_genai import (ChatGoogleGenerativeAI,
                                    HarmBlockThreshold,
                                    HarmCategory)

hug_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
embed_model = LLamaIndexEmbedding(hug_model)
crawler = Crawler()

### Gen outline

In [4]:
outline_doc_creator = OutlineDocumentCreator()
outline_parser = WebPageParser(doc_creator=outline_doc_creator)

model = ChatOpenAI(name='gpt-4-turbo-preview',temperature=0.5)
model_gemini = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.5, safety_settings={
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        })


In [5]:
main_topic = 'thắt ống dẫn tinh'
main_topic_en = 'vasectomy'
use_u2d = True
webpages = crawler.search(main_topic,n=3) # n : số bài top seo lấy về
outlines = []
for page in webpages:
    webpage_doc = WebPageDocument(page)
    outline_parser.feed(webpage_doc)
    outlines.append(outline_parser.parse(in_doc=True))

In [6]:
outlines_headers = ['\n'.join(map(lambda x: x[0].replace('\n',''),o)) for o in outlines if len(o) > 0]

In [7]:
gen_outline_prompt = """Bạn là một bác sĩ đa khoa, bạn có khả năng tạo những dàn ý bài viết đầy đủ, chi tiết cho các chủ đề liên quan y học. Khi tạo dàn ý, bạn cần dựa trên các dàn ý mẫu, các dàn ý mẫu được viết theo dạng <html tag> - <nội dung> , các dàn ý mẫu được phân tách nhau bằng ----, trong các dàn ý mẫu có thể có những mục không liên quan trực tiếp đến bài như 'Bài viết liên quan' 'Liên hệ' hãy bỏ qua các mục này. Nếu các dàn ý mẫu không chứa nội dung liên quan đến chủ đề bài viết, trả lời 'Tôi không biết'. Trong trường hợp các dàn ý mẫu có đề cập đến mục gây chú ý mạnh cho bệnh nhân khiến bệnh nhân muốn đến bệnh viện như 'sự nguy hiểm của bệnh' 'khi nào cần đến bệnh viện' thì hãy thêm vào dàn ý
FORMAT: Dàn ý đầu ra cần tuân thủ chặt chẽ theo format. Mỗi ý một dòng. Không trả lời những thông tin không liên quan đến dàn ý.

DÀN Ý MẪU: {example}

Hãy nêu dàn ý cho bài viết về {topic}
Trả lời:"""

# gen_outline_prompt = """Bạn là một bác sĩ đa khoa, bạn có khả năng tạo những dàn ý bài viết đầy đủ, dễ hiểu cho các chủ đề liên quan y học. Khi tạo dàn ý, bạn cần dựa trên các dàn ý mẫu, trong các dàn ý mẫu có thể có những mục không liên quan trực tiếp đến bài như 'Bài viết liên quan' 'Liên hệ' hãy bỏ qua các mục này. Nếu các dàn ý mẫu không chứa nội dung liên quan đến chủ đề bài viết, trả lời 'Tôi không biết'. Trong trường hợp các dàn ý mẫu có đề cập đến mục gây chú ý mạnh cho bệnh nhân khiến bệnh nhân muốn đến bệnh viện như 'sự nguy hiểm của bệnh' 'khi nào cần đến bệnh viện' thì hãy thêm vào dàn ý

# FORMAT: Dàn ý đầu ra cần tuân thủ chặt chẽ theo format. Mỗi ý một dòng. Không trả lời những thông tin không liên quan đến dàn ý.

# DÀN Ý MẪU: {example}

# Hãy nêu dàn ý cho bài viết về {topic}
# Trả lời:"""

In [8]:
model_res = model.invoke(gen_outline_prompt.format(example='\n----\n'.join(outlines_headers),topic=main_topic))
model_gemini_res = model.invoke(gen_outline_prompt.format(example='\n----\n'.join(outlines_headers),topic=main_topic))

In [9]:
print('outline lấy từ web')
print('\n---\n'.join(outlines_headers))

outline lấy từ web
h1 - Ưu nhược điểm của thắt ống dẫn tinh
h2 - 1. Thắt ống dẫn tinh là gì?
h2 - 2. Đối tượng nào nên thắt ống dẫn tinh
h2 - 3. Ưu nhược điểm của thắt ống dẫn tinh
h3 - 3.1 Ưu điểm của thắt ống dẫn tinh
h3 - 3.2 Nhược điểm của thắt ống dẫn tinh
h2 - 4. Thắt ống dẫn tinh có ảnh hưởng gì không?
h2 - 5. Các biến chứng do thủ thuật thắt ống dẫn tinh
---
h1 - Tin tức
h1 - Thắt ống dẫn tinh là gì? Có đau không? Khi nào nên thực hiện?
h2 - 1. Thắt ống dẫn tinh 
h3 - 1.1. Ống dẫn tinh là gì? 
h3 - 1.2. Thắt ống dẫn tinh là gì?
h2 - 2. Khi nào cần thắt ống dẫn tinh?
h2 - 3. Quy trình thắt ống dẫn tinh
h3 - 3.1. Bác sĩ tư vấn 
h3 - 3.2. Khám tiền thủ thuật
h3 - 3.3. Tiến hành thủ thuật
h2 - 4. Những câu hỏi thường gặp 
h3 - 4.1. Thắt ống dẫn tinh có đau không?
h3 - 4.2. Đàn ông có bị mất ham muốn nếu thắt ống dẫn tinh không?
h3 - 4.3. Sau khi thực hiện thì phải kiêng quan hệ bao lâu?
h2 - Bình luận ()
---
h1 - Thắt ống dẫn tinh: Lợi ích và nguy cơ
h2 - 1. Thắt ống dẫn tinh là gì

In [10]:
org_gen_outline = model_res.content
if len(model_gemini_res.content) < 1000:
    print('outline từ gemini (ko được lấy mặc định)')
    print(model_gemini_res.content)

outline từ gemini (ko được lấy mặc định)
h1 - Thắt ống dẫn tinh: Lợi ích và nguy cơ
h2 - 1. Thắt ống dẫn tinh là gì?
h2 - 2. Lợi ích của thắt ống dẫn tinh
h3 - 2.1. Giảm nguy cơ thai không mong muốn
h3 - 2.2. Giảm nguy cơ lây nhiễm các bệnh lây truyền qua đường tình dục
h3 - 2.3. Cải thiện chất lượng cuộc sống tình dục
h2 - 3. Nguy cơ và rủi ro của thắt ống dẫn tinh
h3 - 3.1. Nguy cơ nhiễm trùng sau thủ thuật
h3 - 3.2. Nguy cơ xuất hiện các vấn đề về tinh trùng
h3 - 3.3. Nguy cơ xuất hiện các biến chứng sau thủ thuật
h2 - 4. Quy trình thực hiện thắt ống dẫn tinh
h3 - 4.1. Chuẩn bị trước thủ thuật
h3 - 4.2. Tiến hành thủ thuật thắt ống dẫn tinh
h3 - 4.3. Hồi phục sau thủ thuật và các biện pháp chăm sóc sau thủ thuật
h2 - 5. Câu hỏi thường gặp về thắt ống dẫn tinh
h3 - 5.1. Thắt ống dẫn tinh có đau không?
h3 - 5.2. Thời gian phục hồi sau thủ thuật là bao lâu?
h3 - 5.3. Thắt ống dẫn tinh có ảnh hưởng đến khả năng sinh sản sau này không?
h2 - 6. Bài viết liên quan
h2 - 7. Liên hệ


In [ ]:
print('outline từ gpt 4 (lấy mặc định)')
print(org_gen_outline)

In [12]:
if input() != 'y':
    raise ValueError('error')

outline từ gpt 4 (lấy mặc định)
h1 - Thắt ống dẫn tinh: Lợi ích và nguy cơ
h2 - 1. Thắt ống dẫn tinh là gì?
h3 - 1.1. Khái niệm về thắt ống dẫn tinh
h3 - 1.2. Mục đích của thắt ống dẫn tinh
h2 - 2. Các yếu tố ảnh hưởng đến quyết định thắt ống dẫn tinh
h3 - 2.1. Đối tượng nào nên thực hiện thắt ống dẫn tinh
h3 - 2.2. Lợi ích của việc thắt ống dẫn tinh
h2 - 3. Quy trình thực hiện thắt ống dẫn tinh
h3 - 3.1. Chuẩn bị trước khi thực hiện thắt ống dẫn tinh
h3 - 3.2. Tiến trình thực hiện thủ thuật
h3 - 3.3. Cách phục hồi sau khi thắt ống dẫn tinh
h2 - 4. Nguy cơ và biến chứng của thắt ống dẫn tinh
h3 - 4.1. Các rủi ro và nguy cơ phát sinh sau thủ thuật
h3 - 4.2. Biến chứng có thể xảy ra sau khi thực hiện thắt ống dẫn tinh
h2 - 5. Thắt ống dẫn tinh và ảnh hưởng đến đời sống tình dục
h3 - 5.1. Tác động của thắt ống dẫn tinh đến ham muốn tình dục
h3 - 5.2. Khả năng phục hồi của chức năng tình dục sau thủ thuật
h2 - 6. Khi nào cần tham khảo ý kiến bác sĩ và quyết định thực hiện thắt ống dẫn tinh

In [13]:
# # Nếu cần lấy outline tùy chỉnh thì sửa ở mục dưới

# org_gen_outline = """h1 - Rối loạn cương dương: Nguyên nhân, triệu chứng, chẩn đoán và điều trị
# h2 - Hình ảnh bệnh Rối loạn cương dương
# h2 - Tổng quan bệnh Rối loạn cương dương
# h2 - Nguyên nhân bệnh Rối loạn cương dương
# h2 - Triệu chứng bệnh Rối loạn cương dương
# h2 - Đối tượng nguy cơ bệnh Rối loạn cương dương
# h2 - Phòng ngừa bệnh Rối loạn cương dương
# h2 - Các biện pháp chẩn đoán bệnh Rối loạn cương dương
# h3 - Khai thác bệnh sử
# h3 - Đánh giá chức năng cương dương
# h3 - Đặt bộ câu hỏi đánh giá
# h3 - Đánh giá mức độ rối loạn cương dương
# h3 - Thăm khám thực thể
# h4 - 3 điểm cần khám lâm sàng:
# h4 - Cận lâm sàng:
# h2 - Các biện pháp điều trị bệnh Rối loạn cương dương
# h3 - Thuốc nhóm ức chế PDE5
# h3 - Sử dụng testosterone
# h3 - Bơm hút chân không
# h3 - Tiêm trực tiếp vào hang vật các thuốc giãn mạch tác động trực tiếp lên dương vật
# h3 - Sử dụng các bài thuốc cổ truyền
# h3 - Phẫu thuật điều trị rối loạn cương dương"""

### Inpage

In [14]:
inpage_parser = WebPageParser(doc_creator=outline_doc_creator)

In [15]:
from big_seo.storage.qdrant import QdrantDB

vec_db = QdrantDB(host='localhost',
                port=6333)
Base.init_instance(vec_db=vec_db)

In [16]:
inpage_parser.parse = partial(inpage_parser.parse, nested=False, in_doc=True,concat_to='h2')
webpage_indexer = InPageIndexer(embedding_model=embed_model,
              parser=inpage_parser,
              Base_model=Base)
for page in webpages:
    page_doc = WebPageDocument(page)
    webpage_indexer.index(page_doc)

In [17]:
# up 2 date
u2d_crawler = UpToDateCrawler()
u2d_parser = UpToDateWebPageParser(doc_creator=outline_doc_creator)
u2d_parser.parse = partial(u2d_parser.parse,nested=False, in_doc=True,concat_to='h2')


translator = GeminiHealthcareTranslator(model='gemini-pro')
translator_parser = TranslationParser(u2d_parser,translator,
                               input_lang=DocumentLang.EN,
                               target_lang=DocumentLang.VI)


translator.translate = partial(translator.translate,other_instruction=f' liên quan đến {main_topic}')
u2d_webpages = u2d_crawler.search(main_topic_en,n=2)
uptodate_indexer = UpToDateInPageIndexer(embedding_model=embed_model,
              parser=translator_parser,
              Base_model=Base)
if use_u2d:
    for page in u2d_webpages:
        page_doc = WebPageDocument(page)
        uptodate_indexer.index(page_doc)

In [ ]:
global_knowledge = set()
def get_gen_knowledge_fn(webpage_indexer, limit):
    def sub(outline):
        prompts = [outline[0]]
        prompts.extend(outline[1])
        docs = set()
        for prompt in prompts:
            retrieved_docs = webpage_indexer.search(prompt)
            docs = docs.union(get_unique_doc(retrieved_docs))
            global_knowledge.update(docs)
            # print(len(docs))
        # return sorted(docs, key=lambda x: x.score, reverse=True)[:5]
        return '\n\n----\n\n'.join(map(lambda x: x.get_doc_meta()['header'] +'\n'+ x.get_doc_content(),sorted(docs, key=lambda x: x.score, reverse=True)[:limit]))
    return sub

In [ ]:
get_knowledge = get_gen_knowledge_fn(webpage_indexer, 4)
if use_u2d:
    get_u2d_knowledge = get_gen_knowledge_fn(uptodate_indexer, 4)

### Gen partial content

#### Prompt template

In [ ]:
partial_prompt = """Bạn là một bác sĩ giàu kinh nghiệm trên lĩnh vực {topic}, dựa trên KIẾN THỨC được cung cấp bạn có khả năng lựa chọn những thông tin phù hợp nhất với chủ đề, bạn cũng có khả năng trình bày theo hướng cung cấp kiến thức chuyên sâu, rõ ràng


BẮT BUỘC: Chỉ được phép cung cấp các thông tin có trong phần KIẾN THỨC, không sử dụng bất cứ thông tin khác .nếu trong phần KIẾN THỨC không chứa nội dung liên quan đến mục bạn đang viết hãy trả lời 'Tôi không biết'


########
KIẾN THỨC: {knowledge}

########
NHIỆM VỤ: hãy viết một đoạn {num_of_words} bằng tiếng Việt về '{headline}' {sub_headline_instruction} Bạn chỉ viết một mục trong bài viết lớn hơn, nên chỉ viết các thông tin liên quan đến '{headline}' không viết các thông tin khác

TRẢ LỜI:"""

h1_prompt = """Bạn là một bác sĩ giàu kinh nghiệm trong lĩnh vực {topic}, dựa vào KIẾN THỨC được cung cấp bạn có thể trình bày lại ngắn gọn dễ hiểu

BẮT BUỘC: nếu trong phần KIẾN THỨC không chứa nội dung liên quan đến mục bạn đang viết hãy trả lời 'Tôi không biết',
Đây là vấn đề liên quan trực tiếp đến sức khỏe, chỉ được phép cung cấp các thông tin có trong phần KIẾN THỨC, không sử dụng bất cứ thông tin khác

########
KIẾN THỨC: {knowledge}

NHIỆM VỤ: Hãy viết một đoạn giới thiệu ngắn từ 200 đến 300 từ về chủ đề {topic} trong đó nêu các phần cụ thể như nguyên nhân, cách điều trị

TRẢ LỜI:"""


rag_prompt = """Bạn là một bác sĩ giàu kinh nghiệm trong lĩnh vực {topic}, dựa trên những KIẾN THỨC được cung cấp, hãy viết một đoạn khoảng 200 từ nêu một số điểm chính liên quan đến {headline}

Không cung cấp các thông tin không có trong phần KIẾN THỨC
Bạn chỉ viết một mục trong bài viết lớn hơn, nên chỉ viết các thông tin liên quan đến '{headline}' không viết các thông tin khác

KIẾN THỨC {knowledge}"""


u2d_enhanced_prompt = """Bạn là một bác sĩ giàu kinh nghiệm trên {topic}, dựa trên KIẾN THỨC được cung cấp bạn có khả năng trình bày theo hướng cung cấp kiến thức chuyên sâu, rõ ràng

BẮT BUỘC: Chỉ được phép cung cấp các thông tin có trong phần KIẾN THỨC, không sử dụng bất cứ thông tin khác .nếu trong phần KIẾN THỨC không chứa nội dung liên quan đến mục bạn đang viết hãy trả lời 'Tôi không biết'


########
KIẾN THỨC: {knowledge}

########
NHIỆM VỤ: Hãy viết lại đoạn sau thành đoạn văn khoảng {num_of_words} về '{headline}' bổ sung thêm các kiến thức liên quan trong phần KIẾN THỨC: {org_content}

TRẢ LỜI:"""


web_normalized_prompt = """Bạn là một bác sĩ giàu kinh nghiệm trên {topic}, dựa trên KIẾN THỨC được cung cấp bạn có khả năng trình bày theo hướng cung cấp kiến thức chuyên sâu, rõ ràng

BẮT BUỘC: Chỉ được phép cung cấp các thông tin có trong phần KIẾN THỨC, không sử dụng bất cứ thông tin khác .nếu trong phần KIẾN THỨC không chứa nội dung liên quan đến mục bạn đang viết hãy trả lời 'Tôi không biết'

########
KIẾN THỨC: {knowledge}

########
NHIỆM VỤ: Hãy viết lại đoạn sau thành đoạn văn khoảng 600 từ về '{headline}' sử dụng những từ ngữ dễ hiểu với người đọc thông thường sử dụng thông tin trong phần KIẾN THỨC: {org_content}

TRẢ LỜI:"""

combine_prompt = """Bạn là một bác sĩ giàu kinh nghiệm trên {topic}, dựa trên KIẾN THỨC được cung cấp bạn có khả năng trình bày theo hướng cung cấp kiến thức chuyên sâu, rõ ràng

BẮT BUỘC: Chỉ được phép cung cấp các thông tin có trong phần KIẾN THỨC, không sử dụng bất cứ thông tin khác .nếu trong phần KIẾN THỨC không chứa nội dung liên quan đến mục bạn đang viết hãy trả lời 'Tôi không biết'
Nếu cả hai bài mẫu đều không đề cập đến {headline} hãy trả lời 'tôi không biết'


Bài mẫu 1: {web_content}



Bài mẫu 2: {u2d_content}

########
NHIỆM VỤ: Hãy kết hợp thông tin từ 2 bài mẫu và viết một đoạn {num_of_words} bằng tiếng Việt về '{headline}'. Bạn chỉ viết một mục trong bài viết lớn hơn, nên chỉ viết các thông tin liên quan đến '{headline}' không viết các thông tin khác

TRẢ LỜI:"""

#### Helper functions

In [ ]:
def get_num_of_words(n_of_outlines, max_n = 400, min_n = 150):
    avg = int((3000/n_of_outlines)/50)*50
    return max(min(avg, max_n), min_n)
def get_better_knowledge(model, outline, topic):
    first_knowledge = get_knowledge(outline=(outline,[]))
    prompt = rag_prompt.format(topic=topic, knowledge=first_knowledge,
                        headline = outline,
                        sub_headline_instruction='')
    # gen rag content
    res = model.invoke(prompt)
    rag_content = res.content

    knowledge = get_knowledge(outline=(outline,[res.content]))
    u2d_knowledge = get_u2d_knowledge(outline=(outline,[res.content]))

    return (rag_content, knowledge, u2d_knowledge)

def gen_content(model,topic,knowledge,headline,sub_headline,n_of_outlines):
    prompt = partial_prompt.format(topic=topic, knowledge=knowledge,
                        headline = headline,
                        sub_headline_instruction=sub_headline,
                        num_of_words=f'từ {get_num_of_words(n_of_outlines)} đến {get_num_of_words(n_of_outlines) + 100} từ')
    return model.invoke(prompt).content

def combine_content(model,topic,
                   web_content,
                   u2d_content,
                   headline,
                   n_of_outlines):
    res = model.invoke(combine_prompt.format(topic=topic,
                   web_content=web_content,
                   u2d_content=u2d_content,
                   headline = headline,
                   num_of_words=f'từ {get_num_of_words(n_of_outlines)} đến {get_num_of_words(n_of_outlines) + 100} từ'))
    return res.content

#### Pipeline

In [ ]:
gen_outlines = org_gen_outline.split('\n')
tracking_results = []
full_gen_content = []
error_knowledge = []
for o in gen_outlines:
    sub_headline = ''
    full_gen_content.append(o)
    try:
        rag_content, knowledge, u2d_knowledge = get_better_knowledge(model_gemini, o, main_topic)
    except:
        rag_content, knowledge, u2d_knowledge = get_better_knowledge(model, o, main_topic)


    # gen content
    prompt = partial_prompt.format(topic=main_topic, knowledge=knowledge,
                            headline = o,
                            sub_headline_instruction=sub_headline,
                            num_of_words=f'từ {get_num_of_words(len(gen_outlines))} đến {get_num_of_words(len(gen_outlines)) + 100} từ')
    web_content = gen_content(model,main_topic,knowledge,o,sub_headline, len(gen_outlines))
    u2d_content = gen_content(model,main_topic,u2d_knowledge,o,sub_headline, len(gen_outlines))
    combined_content = combine_content(model,main_topic,
                    web_content,
                    u2d_content,
                    o,
                    len(gen_outlines))
    tracking_results.append({'outline':o,
                             'rag_content':rag_content,
                             'web_content': web_content,
                             'u2d_content': u2d_content,
                             'knowledge': knowledge,
                             'u2d_knowledge': u2d_knowledge,
                             'combined_content': combined_content
                             })
    if 'tôi không biết' in combined_content.lower():
        print(o)
        error_knowledge.append((o, knowledge))
    else:
        full_gen_content.append(combined_content)


h3 - 3. Lạc nội mạc tử cung có hết sau mãn kinh không?


In [ ]:
# print(rag_content)
# print(web_content)
# print(u2d_content)
# print(combined_content)

#### Process end result

In [ ]:
with open(f'test_output/{main_topic}_raw.txt','w',encoding="utf-8") as f:
    f.write('\n'.join(full_gen_content))
    # f.write('ok')

In [ ]:
final_prompt="""Bạn là một bác sĩ giàu kinh nghiệm trong lĩnh vực {topic}
Dưới đây là một mẫu bài viết về {topic},

Bài mẫu: {example}


YÊU CẦU bạn hãy viết lại bài viết này thành một bài viết chuyên sâu khoảng 2000 từ theo hướng tối ưu chuẩn SEO, đảm bảo mật độ từ khóa chính {topic} tốt.
Với mỗi ý hãy nêu kèm giải thích chọn cách trình bày như liệt kê từng dòng, so sánh hợp lý

BẮT BUỘC: bạn không cung cấp bất cứ kiến thức nào khác ngoài bài mẫu

TRẢ LỜI"""

In [ ]:
# final_res = model_gemini.invoke(final_prompt.format(topic=main_topic,
#                                 outline=org_gen_outline,
#                                 example = '\n'.join(full_gen_content)))
# with open(f'test_output/gemini_rewrite_{main_topic}.txt','w', encoding='utf-8') as f:
#     f.write(final_res.content)

In [ ]:
# final_res = model.invoke(final_prompt.format(topic=main_topic,
#                                              outline=org_gen_outline,
#                                              example = '\n'.join(full_gen_content)))
# with open(f'test_output/gpt_rewrite_{main_topic}.txt','w',encoding='utf-8') as f:
#     f.write(final_res.content)

In [ ]:
# Base.close_instance(delete_model=True)

### Check

In [ ]:
import re
import os
def no_accent_vietnamese(s):
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(r'[Đ]', 'D', s)
    s = re.sub(r'[đ]', 'd', s)
    return s

In [ ]:
# Create new folder
folder_name = 'test_output/' + '-'.join(no_accent_vietnamese(main_topic).split(' '))
if not os.path.exists(folder_name):
    os.mkdir(folder_name)
    os.mkdir(folder_name+'/corr')



#### General check

In [ ]:
if input('general check: ') != 'y':
    raise ValueError('not to check')

In [ ]:
with open('template.txt','r') as f:
    template = f.read()

In [ ]:
data = []
data.append(template.format(_id = 0,
                           cur_state='cur',
                           outline=org_gen_outline.replace('\n','<br>'),
                           model_output='<br>'.join(full_gen_content).replace('\n','<br>'),
                           knowledge='',ud_output='',
                           ud_knowledge='',
                           web_output='',))
for i, point in enumerate(tracking_results):
    u2d_output = point['u2d_content']
    web_output = point['web_content']
    u2d_knowledge = point['u2d_knowledge']
    knowledge = point['knowledge']
    outline = point['outline']
    model_output = point['combined_content']
    _id = i + 1
    cur_state = 'hidden'
    temp = template.format(_id = _id,
                        cur_state=cur_state,
                        outline=outline,
                        model_output=model_output,
                        knowledge=knowledge.replace('\n','<br>'),
                        ud_output=u2d_output,
                        web_output=web_output,
                        ud_knowledge=u2d_knowledge.replace('\n','<br>'))
    data.append(temp)

In [ ]:
with open('index.txt','r') as f:
    index_page = f.read()
index_page = index_page.format(data='\n'.join(data))


with open(folder_name+'/index.html','w', encoding='utf-8') as f:
    f.write(index_page)


os.system(r'copy general_check\scripts.js ' + '\\'.join(folder_name.split('/')) + '\scripts.js')
os.system(r'copy general_check\style.css ' + '\\'.join(folder_name.split('/')) + '\style.css')

os.system(r'copy corr\corr-scripts.js ' + '\\'.join(folder_name.split('/')) + '\corr\corr-scripts.js')
os.system(r'copy corr\corr-style.css ' + '\\'.join(folder_name.split('/')) + '\corr\corr-style.css')

0

In [ ]:
end = datetime.datetime.now()
(end - start).seconds/60

19.9

In [ ]:
# Base.close_instance(delete_model=True)

#### Corr check

In [ ]:
len(full_gen_content)

37

In [ ]:
if input('corr check: ') != 'y':
    raise ValueError('not to check')

In [ ]:
import numpy as np
def get_sentence_vec_from_content(content):
    for sentence in content.split('.'):
        if len(sentence) < 10:
            continue
        vec = embed_model.get_embedding(sentence)
        yield {'vec': np.array(vec),
               'sentence': sentence.replace('\n',' ') + '.'}
def gen_global_knowledge_vec(global_knowledge: set[OutlineDocumentDBType]):
    vecs = []
    for knowledge_doc in global_knowledge:
        knowledge = knowledge_doc.get_doc_content()
        for res in get_sentence_vec_from_content(knowledge):
            vecs.append(res)
    return vecs

def gen_frontend_knowledge(global_knowledge_vecs):
    res = []
    for i in range(len(global_knowledge_vecs)):
        res.append(f'<span id="knowledge-{i}">{global_knowledge_vecs[i]["sentence"]}</span>')
    return '\n'.join(res)

In [ ]:
global_knowledge_vecs = gen_global_knowledge_vec(global_knowledge)
def corr_scale(corr):
    if corr > 0.9:
        return 'veryHigh'
    elif corr > 0.8:
        return 'high'
    elif corr > 0.7:
        return 'med'
    else:
        return None

def calculate_corr(sentence):
    vec = embed_model.get_embedding(sentence)
    res = {'veryHigh':[],
            'high':[],
            'med':[],
            'low':[]}
    for i,k in enumerate(global_knowledge_vecs):
        corr = corr_scale(np.dot(k['vec'], vec))
        if corr is not None:
            res[corr].append(str(i))
    return res

In [ ]:
def process_frontend_gen_content(gen_content):
    res = []
    for sentence in gen_content.split('.'):
        if len(sentence) < 10:
            res.append(f'<span>{sentence}</span>')
        else:
            class_list = 'gen-content'
            attr = ''
            corr = calculate_corr(sentence)
            if len(corr['veryHigh']) > 0:
                class_list += ' veryHigh'
                attr += f' veryHigh="{",".join(corr["veryHigh"][:3])}"'
            if len(corr['high']) > 0:
                class_list += ' high'
                attr += f' high="{",".join(corr["high"][:3])}"'
            if len(corr['med']) > 0:
                class_list += ' med'
                attr += f' med="{",".join(corr["med"][:3])}"'
            res.append(f'<span class="{class_list}" {attr}>{sentence}.</span>')
    return ''.join(res)

In [ ]:
with open('corr-template.txt','r') as f:
    template = f.read()

In [ ]:
template = template.format(gen_content = process_frontend_gen_content('\n'.join(full_gen_content)).replace('\n','<br>'),
global_knowledge=gen_frontend_knowledge(global_knowledge_vecs))

In [ ]:
with open('corr-index.txt','r') as f:
    index_page = f.read()
index_page = index_page.format(data=template)
with open(folder_name+'/corr/index.html','w', encoding='utf-8') as f:
    f.write(index_page)

In [ ]:
Base.close_instance(delete_model=True)

## Curator